# 1. Get query and docuument encoders

In [1]:
import numpy as np

In [12]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

query_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

document_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Sentences we want to encode. Example:

# Sentences are encoded by calling model.encode()

# 2a get dataset

In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset(
    "ms_marco", "v2.1"
)

train_dataset = dataset["train"]

Generating test split: 100%|██████████| 101092/101092 [00:01<00:00, 75834.11 examples/s]


In [8]:
def convert_to_query_pairs(row):
    pairs = []
    for i in range(len(row["passages"]["passage_text"])):
        pairs.append({"query": row["query"], "passage": row["passages"]["passage_text"][i]})
        
    return pairs

In [10]:

# pairs =  convert_to_query_pairs(one_dataset[0])

passages = []
queries = []

filtered_dataset = [item for item in train_dataset if len(item["wellFormedAnswers"]) > 0]

for i in range(len(filtered_dataset)):
    queries.append(filtered_dataset[i]["query"])
    for passage in filtered_dataset[i]["passages"]["passage_text"]:
        passages.append(passage)
        

In [13]:
document_embeddings = document_model.encode(passages)

In [15]:
import pickle
with open("my-embeddings.pkl", "wb") as fOut:
    pickle.dump({'sentences': passages, 'embeddings': document_embeddings},fOut)


In [ ]:
document_dim = document_model.get_sentence_embedding_dimension()
query_dim = query_model.get_sentence_embedding_dimension()

# Apply FAISS


In [ ]:
import faiss